In [ ]:
from pprint import PrettyPrinter

import pandas as pd
from IPython.display import VimeoVideo
from pymongo import MongoClient

In [ ]:
###############Full code for exporting data MongoDB to Python


pp = PrettyPrinter(indent=2)


##Connecting Daatabase network
client = MongoClient(host= "localhost", port= 27017)

#Check number of databases
pp.pprint(list(client.list_databases()))

#select databases
db = client["air-quality"]

#check tables in the databases
for c in db.list_collections():
    print(c["name"])

#select the table
nairobi = db["nairobi"]

#Check different attributes of the table
result = nairobi.find_one({})
pp.pprint(result)

print("Documents from site 6:", nairobi.count_documents({"metadata.site": 6}))
print("Documents from site 29:", nairobi.count_documents({"metadata.site": 29}))

nairobi.distinct("metadata.measurement")


#Check number of observation based on attributes
result = nairobi.aggregate(
   [
       {"$match": {"metadata.site": 6}},
       {"$group": {"_id":"$metadata.measurement","count": {"$count": {}}}}
   ]

)
pp.pprint(list(result))



result = nairobi.aggregate(
   [
       {"$match": {"metadata.site": 29}},
       {"$group": {"_id":"$metadata.measurement","count": {"$count": {}}}}
   ]

)
pp.pprint(list(result))

#Choose the appropiate attribute for using as dataset
result = nairobi.find(
    {"metadata.site": 29, "metadata.measurement": "P2"},
    projection= {"P2": 1, "timestamp": 1, "_id":0}


)
pp.pprint(result.next())



#Import as a dataset
df = pd.DataFrame(result).set_index("timestamp")
df.head()